In [275]:
import re
import numpy as np
import pandas as pd
from wombat.models import Item
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.externals import joblib
from wombat.models import dbsession, engine



plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,10)
%matplotlib inline

# res = find_by_item_type("dresses")
query = """SELECT I.id, I.brand, I.item_type, I.cost,
    I.rent_per_week, I.year_purchased
    FROM items I"""
df = pd.read_sql_query(query, engine)

In [276]:
# Make a list of most frequent brands of the form 'BCBG', 'BCBGMAXAZRIA','Alice + Olivia', etc...
top_brands_query = "SELECT brand, count(brand) FROM items WHERE brand != 'LENDER SUBMISSION FILL IN' GROUP BY brand ORDER BY count(brand) DESC;"
top_brand_df = pd.read_sql_query(top_brands_query, engine)
top_brands = ["\'{}\'".format(brand.replace("'", "''")) for brand in brand_df['brand']]
top_brands = ', '.join(top_brands)

In [277]:
query = "SELECT brand, item_type, cost, rent_per_week FROM items WHERE brand in ({})".format(top_brands)
df = pd.read_sql_query(query, engine)

# get one-hot columns for brands 
dummy_bs = pd.get_dummies(df['brand'])
dummified_df = pd.concat([df, dummy_bs], axis = 1)
dummified_df = dummified_df.drop('brand', axis = 1)

# get one-hot columns for brands
dummified_items = pd.get_dummies(df['item_type'])
dummified_items = pd.concat([dummified_df, dummified_items], axis = 1)
df = dummified_items.drop('item_type', axis = 1)

In [279]:
# split data into train and test group
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]

print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:', len(test))

Number of observations in the training data: 4511
Number of observations in the test data: 1493


In [280]:
y = train['rent_per_week']
features = df.columns.drop('rent_per_week') # just column names

reg = linear_model.LinearRegression()
reg.fit(train[features], train['rent_per_week'])
#joblib.dump(reg, 'filename.pkl')
#call this with: clf = joblib.load('filename.pkl') 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [281]:
test['predicted_rent'] = reg.predict(test[features])
def r2 (row):
   return row['predicted_rent'] - row['rent_per_week']
test['rent_difference'] = test.apply(lambda row: r2(row), axis=1)
reg.score(train[features], train['rent_per_week'])

/home/patrick/.virtualenvs/stylelend/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/patrick/.virtualenvs/stylelend/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0.51706171862775596

In [282]:
clf = RandomForestRegressor()
clf.fit(train[features], train['rent_per_week'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [284]:
rfr_pred = clf.predict(test[features]).astype(int)

In [288]:
#sklearn.metrics.confusion_matrix(test[features.drop('is_train')], rfr_pred)
test['predicted_rent'] = reg.predict(test[features])
def r2 (row):
   return row['predicted_rent'] - row['rent_per_week']
test['rent_difference'] = test.apply(lambda row: r2(row), axis=1)
df2 = test[features.drop('is_train')]
test['rent_difference']

/home/patrick/.virtualenvs/stylelend/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/patrick/.virtualenvs/stylelend/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


2       5.438450e+04
3       3.390049e+04
4       8.343375e+03
12      5.402344e+01
17      7.914972e+10
24      7.914971e+10
25      2.318492e+03
27      8.346094e+02
29      3.233294e+04
39      2.826289e+03
43      7.914964e+10
48      9.339879e+04
49     -1.834664e+04
51      7.860938e+01
53     -5.310234e+02
54      3.859297e+02
55     -2.237031e+02
61     -8.333359e+02
67     -7.254688e+01
68      1.142645e+05
73      1.106922e+05
76      1.122138e+05
84     -4.670422e+03
87      8.462578e+02
100     7.257188e+02
105     5.234172e+03
106     3.593227e+03
120     6.453594e+03
121    -4.798598e+04
124    -4.999983e+04
            ...     
5889    7.914972e+10
5890    2.470800e+04
5898   -5.689844e+02
5899   -6.927695e+03
5904    2.448491e+04
5905    2.433176e+04
5912    6.312305e+03
5918    8.346110e+04
5921    2.554331e+13
5922   -8.816719e+02
5929    2.302207e+04
5930    2.265625e-01
5933   -2.879750e+03
5937    3.254930e+04
5938   -1.763406e+03
5939    5.460711e+03
5941    8.126

In [271]:
# try random forest regression without dummifying variables first
# this ended up not working. It said, "Cannot convert string to float"
# which leads me to believe it was looking for a number instead of a category
# this means the original way of one-hoting the categories was correct.
df = pd.read_sql_query(query, engine)
# split data into train and test group
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]

print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:', len(test))

y = train['rent_per_week']
features = df.columns.drop('rent_per_week') # just column names

reg = linear_model.LinearRegression()
reg.fit(train[features], train['rent_per_week'])

Number of observations in the training data: 4697
Number of observations in the test data: 1571


ValueError: could not convert string to float: 'Alice + Olivia'